In [ ]:
import tensorflow.compat.v1 as tf
#Lets see if tensorflow finds the GPU
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
#import tensorflow as tf

In [ ]:
#Lets see if it works
tf.ones(1) + tf.ones(1)

In [ ]:
import numpy as np # for working with arrays and matrices
import pandas as pd # for data manipulation and analysis
import matplotlib.pyplot as plt # for data visualization
import seaborn as sns # for data visualization

import os # OS module provides various operating system-related functions to the code
import csv # CSV module is used for working with CSV (Comma Separated Values) files in Python.
import pickle



# used for splitting data into training and testing sets in Python.
from sklearn.model_selection import train_test_split 

# for generating a confusion matrix
from sklearn.metrics import confusion_matrix
from keras.models import load_model


# Classes and functions from the Keras library which is used for building and training deep learning models in Python.
from keras.models import model_from_json
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dropout
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# These import the Adam optimizer class and various other classes from the TensorFlow Keras library 
# which is a high-level neural networks API used for building and training deep learning models in Python.
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
# Load the history object from the pickle file
with open('history.pkl', 'rb') as f:
    history = pickle.load(f)
    
with open('train_images.pkl', 'rb') as f:
    train_images = pickle.load(f)
    
with open('train_labels.pkl', 'rb') as f:
    train_labels = pickle.load(f)

In [ ]:
# Inspect the contents of the pickle file
print(history)

In [ ]:
# plot the accuracy and loss of the model during training. (validation later)
def plot_accuracy():
    plt.plot(history['accuracy'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train'], loc='upper left')
    plt.savefig('accuracy.png')
    return plt.show()

def plot_loss():
    plt.plot(history['loss'])
    plt.title('Model loss')
    plt.ylabel('loss')
    plt.xlabel('Epoch')
    plt.legend(['Train'], loc='upper right')
    plt.savefig('loss.png')
    return plt.show()

def plot_cm(no_of_behaviors, confusion):
    

    # Plot the confusion matrix
    fig, ax = plt.subplots(figsize=(8,6))
    cmap = plt.get_cmap('Blues')

    tick_marks = np.arange(len(no_of_behaviors))
    plt.xticks(tick_marks, no_of_behaviors, rotation=45)
    plt.yticks(tick_marks, no_of_behaviors)
    plt.tight_layout()

    ax.imshow(confusion, interpolation='nearest', cmap=cmap)
    ax.set_xticks(np.arange(len(no_of_behaviors)))
    ax.set_yticks(np.arange(len(no_of_behaviors)))
    ax.set_xticklabels(no_of_behaviors)
    ax.set_yticklabels(no_of_behaviors)
    ax.set_xlabel('Predicted labels')
    ax.set_ylabel('True labels')

    # cm = confusion_matrix (train_labels, predicted_labels)
    # print(cm)

    # Add text annotations to the confusion matrix boxes
    for i in range(len(no_of_behaviors)):
        for j in range(len(no_of_behaviors)):
            ax.text(j, i, str(f'{(confusion[i, j]/counts[i])*100:.2f}%'), ha='center', va='center', color='black')

# Set the title
    plt.savefig('cm.png')
    ax.set_title('Confusion Matrix')
    return plt.show()

In [ ]:
# format the train_labels to be appropriate for the predict method
train_labels = np.argmax(train_labels, axis=1)

# Predict the class labels of the training images using the trained model
model = load_model('BPNN_V1_model.h5')
predicted_labels = np.argmax(model.predict(train_images), axis=1)

# Compute the confusion matrix using the predicted class labels and the true class labels
confusion = confusion_matrix(train_labels, predicted_labels)
no_of_behaviors = ['Main Corr', 'Left Corr', 'Right Corr']


In [ ]:
main = 11336
left = 9643
right = 3610
total_counts = main + left + right
counts = [main, left, right]

In [ ]:
plot_accuracy()
plot_loss()
plot_cm(no_of_behaviors, confusion)